In [1]:
import numpy as np
import pandas as pd
import math
from scipy import stats
import yfinance as yf
from statistics import mean

In [2]:
sp500 = pd.read_csv("data/sp500.csv")

In [3]:
sp500.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [4]:
tickers = sp500['Symbol'].tolist()

In [5]:
tick = []
price = []
month1 = []
month3 = []
month6 = []
month12 = []

In [6]:
for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period="1y")
        if len(hist) < 200:
            print(ticker)
            continue
        current_price = hist['Close'].iloc[-1]
        tick.append(ticker)
        price.append(current_price)
        
        # Calculate monthly returns
        month1.append(current_price / hist['Close'].iloc[-21] - 1)
        month3.append(current_price / hist['Close'].iloc[-63] - 1)
        month6.append(current_price / hist['Close'].iloc[-126] - 1)
        month12.append(current_price / hist['Close'].iloc[-len(hist)] - 1)
    except:
        continue

HTTP Error 404: {"quoteSummary":{"result":null,"error":{"code":"Not Found","description":"Quote not found for symbol: ANSS"}}}
$ANSS: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")


ANSS


$BRK.B: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")


BRK.B


$BF.B: possibly delisted; no price data found  (period=1y)


...

HES


$PARA: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")


PARA


$WBA: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")


WBA


In [7]:
momentum_data = {
    'Ticker': tick,
    'Price': price,
    '1_month': month1,
    '3_month': month3,
    '6_month': month6,
    '12_month': month12
}

In [8]:
mdf = pd.DataFrame(momentum_data)

In [9]:
mdf.head()

,Ticker,Price,1_month,3_month,6_month,12_month
0,MMM,164.350006,0.058070,0.088107,0.196755,0.251667
1,AOS,66.044998,-0.065662,-0.062496,-0.018914,-0.115544
2,ABT,124.540001,-0.068341,-0.068411,-0.062274,0.086471
3,ABBV,216.785004,-0.055752,0.102488,0.186732,0.115134
4,ACN,247.839996,-0.020318,0.036120,-0.193604,-0.294139


In [10]:
# Calculate annualized daily volatility
annVol = []
for ticker in tick:
    stock = yf.Ticker(ticker)
    hist = stock.history(period="1y")
    dailyReturns = []

    # Calculate daily returns
    for i in range(1, len(hist)):
        today = hist['Close'].iloc[i]
        yesterday = hist['Close'].iloc[i-1]
        dailyReturns.append(today/yesterday - 1)
    # Calculate daily volatility
    dailyVolatility = np.std(dailyReturns, ddof=1)
    # Annualize daily volatility
    annVol.append(dailyVolatility * math.sqrt(252))

In [11]:
mdf["Daily_Vol"] = annVol

In [12]:
mdf.to_csv("sp500_HQM.csv", index=False)